## Libraries

In [126]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pmdarima import arima
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from pathlib import Path

In [127]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [128]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [129]:
from pathlib import Path

In [130]:
%run ../Libraries/SVRmape_library.ipynb
%run ../Utils/Utilss.ipynb

## Load data

In [131]:
pathtofile = Path().resolve().parent / "Data" / "Data1.csv"
df = pd.read_csv(pathtofile)[["fecha", "MWh"]].dropna()

In [132]:
df["fecha"] = pd.to_datetime(df["fecha"], format = "%m/%d/%Y")
df.set_index("fecha", inplace = True)
df = df.groupby(pd.Grouper(freq='MS')).agg({"MWh":"mean"})

In [133]:
train = df.query("index < 2018")
test = df.query("index > '2017-12-31'")

In [134]:
df_trf = Utilss.transform_diflog(train, offset = 1)

In [135]:
lag_order = 4
horizon = df.size - df_trf.size

In [136]:
# embed matrix
mbd = Utilss.embed(df_trf, lag_order)

In [137]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test1 = Utilss.train_test(mbd, df, horizon)

In [138]:
y_test = y_test1.to_numpy().reshape(-1)

## Optimization function

In [145]:
def opt_bas(C, epsilon, gamma):
    
    # parameters
    hyperparameters = {
#         'alpha' : alpha,
#         'max_iter':10000,
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon,
        'gamma' : gamma,
#         "n_estimators" : int(n_estimators),
#         "max_depth" : 3,
#         "eta" : eta,
#         "objective": "reg:squarederror",
    }
    
    # fit and predict
    predict = Utilss.forecasting(SVR_mape, X_train, y_train, **hyperparameters)
    
    y_pred = Utilss.backtransform(predict, train, 1)
    
    # get score
    mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
    
    return -mape

## Run optimization

In [146]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [147]:
# Bounded region of parameter space
# pbounds = {'n_estimators': (120, 125), 'eta': (0.01, 0.1)}
pbounds = {'C': (0.01, 10), 'epsilon': (0.01, 0.10), 'gamma': (0.01, 10)}
# pbounds = {'alpha':(0.001, 100)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "SVR_mape.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [148]:
optimizer.maximize(init_points=10, n_iter=100)